In [1]:
import numpy as np
# import torch
import matplotlib.pyplot as plt
import pandas as pd  

In [3]:
data_file = '../../../data/Beijing/4x6x12x6/beijing_15k.npy'
# data_file = '../data/'
raw_data = np.load(data_file, allow_pickle=True).item()
print(raw_data.get('ndims'))
data = raw_data.get('data')


[12, 6, 12]


In [4]:
sum(raw_data.get('ndims'))

30

In [5]:
fold = 0
tr_ind = data[0]['tr_ind']
tr_y = data[0]['tr_y']
tr_T = data[0]['tr_T']

te_ind = data[0]['te_ind']
te_y = data[0]['te_y']
te_T = data[0]['te_T']

all_data = pd.DataFrame()
all_ind = np.concatenate((tr_ind,te_ind))
all_time = np.concatenate((tr_T,te_T)) # zheng has alreadly normalized the time into 0-1
all_y = np.concatenate((tr_y,te_y))

all_data['location'] = all_ind[:,0].astype(np.int64)
all_data['pollutant'] = all_ind[:,1].astype(np.int64)
all_data['period'] = all_ind[:,2].astype(np.int64)
all_data['y'] = all_y.astype(float)
all_data['time'] = all_time


time_uni_all = np.sort(np.unique(all_time))
time_uni_tr = np.sort(np.unique(all_time))

if len(time_uni_all)==len(time_uni_tr):
    print('no never-seen time-stamp in test data')
else: print('exist never-seen time-stamp in test data')

no never-seen time-stamp in test data


In [6]:
def unique_recoding(data,colum_name):
    # colum_name = 'movieId'
    unique_key = np.sort(data[colum_name].unique())
    recode_dict = {key:id for id,key in enumerate(unique_key)}

    new_column = data[colum_name].apply(lambda x:recode_dict[x])
    # data[colum_name] = new_column
    print('ndim of %s is %d'%(colum_name,len(new_column.unique())))
    return recode_dict, new_column

def unique_recoding_array(data_array):
    
    unique_key = np.unique(data_array)
    recode_dict = {key:id for id,key in enumerate(unique_key)}

    new_array =  np.array(list(map(lambda x:recode_dict[x], data_array)))
    # data[colum_name] = new_column
    # print('ndim of %s is %d'%(colum_name,len(new_column.unique())))
    return new_array

recode_dict_globle,all_data['timestamp_disct'] = unique_recoding(all_data,'time') 
# use for baseline, where we simply encoding the timastamp as extra mode 

ndim of time is 1461


In [7]:
data_save = {}
data_save['ndims'] = raw_data.get('ndims')
data_save['num_node'] = np.sum(data_save['ndims'])
data_save['data'] = []
data_save['time_uni'] = np.sort(np.unique(all_time)) # sorted unique timestamps of all data
data_save['time_id_table'] = recode_dict_globle # timestamp-disct id map table

for fold in range(5):

    fold_data = {}
    tr_ind = data[fold]['tr_ind']
    tr_y = data[fold]['tr_y']
    tr_T = data[fold]['tr_T']

    te_ind = data[fold]['te_ind']
    te_y = data[fold]['te_y']
    te_T = data[fold]['te_T']

    # we have to sort the training data by the time-stamp
    # also sort the test data, but it's not necessary
    tr_sort_id = np.argsort(np.squeeze(tr_T))
    fold_data['ind_tr'] = tr_ind[tr_sort_id,:].astype(int)
    fold_data['y_tr'] = tr_y[tr_sort_id]
    fold_data['T_tr'] = tr_T[tr_sort_id]

    te_sort_id = np.argsort(np.squeeze(te_T))
    fold_data['ind_te'] = te_ind[te_sort_id,:].astype(int)
    fold_data['y_te'] = te_y[te_sort_id]
    fold_data['T_te'] = te_T[te_sort_id]

    # we discrete the time-stamps of all data(both train and test),and encoding them with id
    # there are two usages of this feature:
    # 1: use them as extra mode in some non-dynamict baselines  
    # 2: to efficient identify the group of obseved entries with same time-stamps

    fold_data['T_disct_tr'] = np.array(list(map(lambda x:recode_dict_globle[x], np.squeeze(fold_data['T_tr'])))).astype(int)
    fold_data['T_disct_te'] = np.array(list(map(lambda x:recode_dict_globle[x], np.squeeze(fold_data['T_te'])))).astype(int)

    data_save['data'].append(fold_data)

    
file_name = '../../processed_data/beijing_15k.npy'
np.save(file_name, data_save)

In [14]:
data_save["time_uni"][1]-data_save["time_uni"][0]

0.0006849315068493151